In [1]:
import numpy as np
import pandas as pd
import glob
from os.path import join as pjoin
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [149]:
icapath = '/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodretinotopy/mfm_locwise_fullpipeline/anal/feature-ica'
components = []
for i in range(9):
    for ses in range(4):
        components.append(joblib.load(pjoin(icapath, f'sub-0{i}_ses-{ses+1}_ica_model.pkl')).components_)
components = np.vstack(components)
compcors = np.corrcoef(components)

In [68]:
selected = [8, 21, 33, 39, 44, 45, 51, 5, 10, 11, 31, 48, 73, 149, 266, 56, 78, 86]

In [150]:
n_feature = 63
compidxs = np.arange(compcors.shape[0])
threses = [0.99, 0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6]
selected = []
for thres in threses:
    print(thres)
    for _ in range(9):
        highcorr_compnum = np.array([np.sum((np.abs(compcors[_])) > thres) for _ in range(compcors.shape[0])])
        comps = np.where( highcorr_compnum == 36)[0]
        if len(comps) > 0:
            related_cols = np.argsort(np.abs(compcors[comps[0]]))[::-1][0:36]
            for comp in comps:
                if  np.all(np.sort( related_cols // n_feature) == np.arange(36)):
                    print(compidxs[comp], '每个被试都有:', np.all(np.sort( related_cols // n_feature) == np.arange(36)))
                    selected.append(compidxs[comp])
                    n_feature -= 1 
                    compcors = np.delete(compcors, related_cols, axis=0)
                    compcors = np.delete(compcors, related_cols, axis=1)
                    compidxs = np.delete(compidxs, related_cols)
                    
                    break

0.99
8 每个被试都有: True
21 每个被试都有: True
33 每个被试都有: True
39 每个被试都有: True
44 每个被试都有: True
45 每个被试都有: True
51 每个被试都有: True
0.95
5 每个被试都有: True
10 每个被试都有: True
11 每个被试都有: True
31 每个被试都有: True
48 每个被试都有: True
73 每个被试都有: True
149 每个被试都有: True
266 每个被试都有: True
0.9
56 每个被试都有: True
78 每个被试都有: True
86 每个被试都有: True
0.85
67 每个被试都有: True
1601 每个被试都有: True
0.8
4 每个被试都有: True
686 每个被试都有: True
1059 每个被试都有: True
1522 每个被试都有: True
1720 每个被试都有: True
0.75
414 每个被试都有: True
1950 每个被试都有: True
2177 每个被试都有: True
0.7
372 每个被试都有: True
1612 每个被试都有: True
0.65
0.6


In [152]:
import numpy as np
from sklearn.cluster import KMeans

# 假设你的数据已经加载到变量 data 中，且 data 的形状为 (1620, 1620)
# data = np.load('your_data_file.npy')  # 例如，如果数据存储在.npy文件中
data = np.abs(compcors)
# 初始化KMeans模型，并进行拟合
kmeans = KMeans(n_clusters=n_feature, random_state=0)
kmeans.fit(data)

# 获取聚类中心和标签
centers = kmeans.cluster_centers_
labels = kmeans.labels_

# 初始化一个列表来存储每个聚类中心最近的样本索引
closest_indices = []

# 遍历每个聚类中心
for i in range(len(centers)):
    # 找到属于当前聚类的所有点
    members = data[labels == i]
    
    # 计算这些点到聚类中心的距离
    distances = np.linalg.norm(members - centers[i], axis=1)
    
    # 找到距离最小的点的索引
    closest_index = np.argmin(distances)
    
    # 将属于当前聚类的所有点中距离最近的那个点的原始索引添加到列表中
    # closest_samples.append(np.where(labels == i)[0][closest_index])
    closest_indices.append(np.where(labels == i)[0][closest_index])
# 打印每个聚类中心最近的样本的索引
print("Indices of closest samples to each cluster center:", compidxs[np.array(closest_indices)])


Indices of closest samples to each cluster center: [  23 1659 2092   32 2027  490 1375 1381  181  255 1108 1377  173 1088
 1112 1684  186 2137 2134 1125 2114  273 2156 2041 1928 1141  358 1102
 1527 1094 1666  497 2080]


In [155]:
selected.extend(compidxs[np.array(closest_indices)])
np.array(selected)

array([   8,   21,   33,   39,   44,   45,   51,    5,   10,   11,   31,
         48,   73,  149,  266,   56,   78,   86,   67, 1601,    4,  686,
       1059, 1522, 1720,  414, 1950, 2177,  372, 1612,   23, 1659, 2092,
         32, 2027,  490, 1375, 1381,  181,  255, 1108, 1377,  173, 1088,
       1112, 1684,  186, 2137, 2134, 1125, 2114,  273, 2156, 2041, 1928,
       1141,  358, 1102, 1527, 1094, 1666,  497, 2080])

In [158]:
np.save(pjoin(icapath, 'ica-unmixing_consistency-kmeans-centers.npy'), components[np.array(selected)])

In [159]:
unmixing = components[np.array(selected)]

In [129]:
import numpy as np
from sklearn.cluster import KMeans

# 假设你的数据已经加载到变量 data 中，且 data 的形状为 (1620, 1620)
# data = np.load('your_data_file.npy')  # 例如，如果数据存储在.npy文件中
data = np.abs(compcors)
# 初始化KMeans模型，并进行拟合
kmeans = KMeans(n_clusters=63, random_state=0)
kmeans.fit(data)

# 获取聚类中心和标签
centers = kmeans.cluster_centers_
labels = kmeans.labels_

# 初始化一个列表来存储每个聚类中心最近的样本索引
closest_indices = []

# 遍历每个聚类中心
for i in range(len(centers)):
    # 找到属于当前聚类的所有点
    members = data[labels == i]
    
    # 计算这些点到聚类中心的距离
    distances = np.linalg.norm(members - centers[i], axis=1)
    
    # 找到距离最小的点的索引
    closest_index = np.argmin(distances)
    
    # 将属于当前聚类的所有点中距离最近的那个点的原始索引添加到列表中
    # closest_samples.append(np.where(labels == i)[0][closest_index])
    closest_indices.append(np.where(labels == i)[0][closest_index])
# 打印每个聚类中心最近的样本的索引
print("Indices of closest samples to each cluster center:", closest_indices)


Indices of closest samples to each cluster center: [172, 1291, 2137, 273, 255, 117, 1373, 1807, 1928, 1098, 2228, 318, 1365, 2209, 10, 639, 2112, 620, 345, 1815, 1330, 1298, 2144, 582, 1852, 2139, 2225, 1663, 1288, 1381, 2125, 1766, 1272, 59, 186, 1405, 1112, 1858, 1207, 1349, 1375, 2041, 2140, 1802, 173, 1088, 1084, 1094, 2092, 1659, 2114, 1141, 1091, 29, 2027, 2134, 426, 291, 1950, 147, 497, 1377, 490]


In [136]:
np.save(pjoin(icapath, 'ica-unmixing_kmeans-centers.npy'), components[closest_indices])

In [135]:
np.unique(closest_indices) // 63

array([ 0,  0,  0,  1,  2,  2,  2,  2,  4,  4,  4,  5,  5,  6,  7,  7,  9,
        9, 10, 17, 17, 17, 17, 17, 17, 18, 19, 20, 20, 20, 20, 21, 21, 21,
       21, 21, 21, 21, 22, 26, 26, 28, 28, 28, 28, 29, 29, 30, 30, 32, 32,
       33, 33, 33, 33, 33, 33, 33, 33, 34, 35, 35, 35])